In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [3]:
df = pd.read_csv("~/Downloads/snli_1.0/snli_1.0_train.csv", sep='\t')
df.head()

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
0,neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,NaN,NaN,NaN,NaN
1,contradiction,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4,3416050480.jpg#4r1c,contradiction,NaN,NaN,NaN,NaN
2,entailment,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,"( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4,3416050480.jpg#4r1e,entailment,NaN,NaN,NaN,NaN
3,neutral,( Children ( ( ( smiling and ) waving ) ( at c...,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2,2267923837.jpg#2r1n,neutral,NaN,NaN,NaN,NaN
4,entailment,( Children ( ( ( smiling and ) waving ) ( at c...,( There ( ( are children ) present ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,Children smiling and waving at camera,There are children present,2267923837.jpg#2,2267923837.jpg#2r1e,entailment,NaN,NaN,NaN,NaN


In [4]:
df.shape

(550152, 14)

In [5]:

df.dropna(subset = ["sentence2"], inplace=True)

In [6]:
df.shape

(550146, 14)

In [7]:
df = df.reset_index(drop=True)

In [9]:

from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_multiple_whitespaces, remove_stopwords, stem_text
custom_filters = [strip_tags, strip_multiple_whitespaces, remove_stopwords, stem_text]
def get_tokenized_questions(X):
    series = pd.Series(pd.concat([X['sentence1'], X['sentence2']]),dtype=str)
    series.dropna()
    for question in series:
        yield preprocess_string(question, custom_filters)

In [10]:
# preprocess the questions in the training set 
X_train_tokens = get_tokenized_questions(df)

In [10]:
import gensim
from gensim.models import KeyedVectors


# step 1: initialize word2vec model with the vocabulary of training data
model_w2v = gensim.models.Word2Vec(X_train_tokens, size=300) 

# step 2: intersect the initialized word2vec model with the pre-trained fasttext model
model_w2v.intersect_word2vec_format(input_folder+'GoogleNews-vectors-negative300.bin',
                                lockf=1.0,
                                binary=True)

# step 3: improve model with transfer-learning using the training data
model_w2v.train(X_train_tokens,total_examples=model_w2v.corpus_count, epochs=10)

TypeError: You can't pass a generator as the sentences argument. Try a sequence.

In [11]:
from gensim.models import KeyedVectors
# load the google word2vec model
filename = '/Users/mariamma/Documents/coursework/deep_learning/assignment3/GoogleNews-vectors-negative300.bin'
model_w2v = KeyedVectors.load_word2vec_format(filename, binary=True)
# calculate: (king - man) + woman = ?
result = model_w2v.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

[('queen', 0.7118192911148071)]


In [22]:

result = model_w2v.most_similar(positive=['Soviet'], topn=1)
print(result)

[('Soviet_Union', 0.8393017649650574)]


In [11]:
from gensim.parsing.preprocessing import preprocess_string, strip_tags,strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, stem_text
custom_filters = [strip_tags,strip_punctuation,strip_numeric, strip_multiple_whitespaces, remove_stopwords, stem_text]
def get_tokenized_questions(X):
    tokenlist = []
    series = pd.Series(pd.concat([X['sentence1'], X['sentence2']]),dtype=str)
    series.dropna()
    for question in series:
        res =  preprocess_string(question, custom_filters)
        tokenlist.append(res)
    return tokenlist

In [12]:
# preprocess the questions in the training set 
X_train_tokens = get_tokenized_questions(df)

In [13]:
X_train_tokens

[['a', 'person', 'hors', 'jump', 'broken', 'airplan'],
 ['a', 'person', 'hors', 'jump', 'broken', 'airplan'],
 ['a', 'person', 'hors', 'jump', 'broken', 'airplan'],
 ['children', 'smile', 'wave', 'camera'],
 ['children', 'smile', 'wave', 'camera'],
 ['children', 'smile', 'wave', 'camera'],
 ['a', 'boi', 'jump', 'skateboard', 'middl', 'red', 'bridg'],
 ['a', 'boi', 'jump', 'skateboard', 'middl', 'red', 'bridg'],
 ['a', 'boi', 'jump', 'skateboard', 'middl', 'red', 'bridg'],
 ['an',
  'older',
  'man',
  'sit',
  'orang',
  'juic',
  'small',
  'tabl',
  'coffe',
  'shop',
  'employe',
  'bright',
  'color',
  'shirt',
  'smile',
  'background'],
 ['an',
  'older',
  'man',
  'sit',
  'orang',
  'juic',
  'small',
  'tabl',
  'coffe',
  'shop',
  'employe',
  'bright',
  'color',
  'shirt',
  'smile',
  'background'],
 ['an',
  'older',
  'man',
  'sit',
  'orang',
  'juic',
  'small',
  'tabl',
  'coffe',
  'shop',
  'employe',
  'bright',
  'color',
  'shirt',
  'smile',
  'background']

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
pass_through = lambda x:x
tfidf = TfidfVectorizer(analyzer=pass_through)
# compute tf-idf weights for the words in the training set questions
X_tfidf = tfidf.fit_transform(X_train_tokens)

In [15]:
print(tfidf.get_feature_names())

['a', 'aa', 'aaa', 'aaah', 'aad', 'aan', 'aar', 'aardvark', 'aaron', 'aashni', 'aasian', 'ab', 'aback', 'abalon', 'aband', 'abandon', 'abarrot', 'abba', 'abbei', 'abbot', 'abbout', 'abbrevi', 'abc', 'abdomen', 'abdomin', 'abduct', 'abductor', 'abeisl', 'abercrombi', 'aberdeen', 'abesto', 'abil', 'abl', 'ablaz', 'abnei', 'aboard', 'abod', 'aboiut', 'abolish', 'abook', 'aborigin', 'abort', 'abou', 'abound', 'about', 'abouv', 'abov', 'aboveground', 'abowl', 'abr', 'abraham', 'abreak', 'abreast', 'abroad', 'abruptli', 'abseil', 'absenc', 'absent', 'absolut', 'absoluteili', 'absorb', 'abstain', 'abstic', 'abstract', 'abstractli', 'abu', 'abubakar', 'abucket', 'abuild', 'abund', 'abus', 'abut', 'ac', 'academ', 'academi', 'acapella', 'acapello', 'acapulco', 'accapella', 'acceler', 'accend', 'accent', 'accentu', 'accept', 'accesori', 'access', 'accessor', 'accessori', 'accid', 'acciden', 'accident', 'accidentali', 'accommod', 'accompani', 'accomplish', 'accord', 'accordian', 'accordion', 'acco

In [16]:
# split into two
# X1_tfidf -> tf-idf weights of first question in question pair and 
# X2_tfidf -> tf-idf weights of second question in question pair
X1_tfidf = X_tfidf[:len(df)]
X2_tfidf = X_tfidf[len(df):]

In [17]:
X1_tfidf

<550146x21341 sparse matrix of type '<class 'numpy.float64'>'
	with 4115229 stored elements in Compressed Sparse Row format>

In [18]:
df.shape

(550146, 14)

In [19]:
feature_names = tfidf.get_feature_names()

In [20]:
doc = 550146
feature_index = X_tfidf[doc,:].nonzero()[1]

In [21]:
feature_index

array([ 3891, 19252,  8879, 13493,     0], dtype=int32)

In [22]:
feature_names[3891]

'competit'

In [23]:

tfidf_scores = zip(feature_index, [X_tfidf[doc, x] for x in feature_index])

In [24]:
tfidf_scores

In [25]:
for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
  print(w, s)

competit 0.5863431120511333
train 0.5106152188108466
hors 0.4844080257942244
person 0.3727515297799401
a 0.14791556666490513


In [27]:
import gensim
from gensim.models import KeyedVectors

model_w2v2 = gensim.models.Word2Vec(X_train_tokens, size=300) 

input_folder = '/Users/mariamma/Documents/coursework/deep_learning/assignment3/'
# step 2: intersect the initialized word2vec model with the pre-trained fasttext model
model_w2v2.intersect_word2vec_format(input_folder+'GoogleNews-vectors-negative300.bin',
                                lockf=1.0,
                                binary=True)

# step 3: improve model with transfer-learning using the training data
model_w2v2.train(X_train_tokens,total_examples=model_w2v2.corpus_count, epochs=10)

(48630581, 67318530)

In [28]:

df.shape[0]

550146

In [29]:
from gensim.parsing.preprocessing import preprocess_string, strip_tags,strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, stem_text
custom_filters = [strip_tags,strip_punctuation,strip_numeric, strip_multiple_whitespaces, remove_stopwords, stem_text]
def tokenize_sentence(X):
    res =  preprocess_string(X, custom_filters)
    return res

In [30]:
sentence_new = tokenize_sentence("A person on a horse jumps over a broken down airplane.")
sentence_new

['a', 'person', 'hors', 'jump', 'broken', 'airplan']

In [31]:
def findTfIdf(row_num, X_tfidf, feature_names):
    sent1_index = 0 + row_num
    sent2_index = df.shape[0] + row_num
    tfidfdict = {}
    
    doc = sent1_index
    feature_index = X_tfidf[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [X_tfidf[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
        tfidfdict[w]=s
        
    doc = sent2_index
    feature_index = X_tfidf[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [X_tfidf[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
        tfidfdict[w]=s    
    return tfidfdict   

In [32]:
dt = findTfIdf(4, X_tfidf,feature_names)
dt

{'camera': 0.4998957607118208,
 'wave': 0.5705740417354377,
 'smile': 0.4920773485220348,
 'children': 0.4819227484640764,
 'present': 0.7334840497339453,
 'there': 0.47932412134037417}

In [33]:
def findSentenceCorrelation(row, model_w2v2, X_tfidf, feature_names,word_vectors):
    #print(row.name)
    tfidf_dict = findTfIdf(row.name,X_tfidf, feature_names)
    #print(tfidf_dict)
    sent1 = tokenize_sentence(row['sentence1'])
    sent2 = tokenize_sentence(row['sentence2'])
    #print(sent1, sent2)
    sim = 0
    for word1 in sent1:
        for word2 in sent2:
            if word1 in word_vectors.vocab:
                if word2 in word_vectors.vocab:
                    #print("Row: ", row.name, " word1: ", word1, " word2: ", word2, " sim_score: ", model_w2v2.similarity(word1, word2), " tfidf : ",  tfidf_dict[word1], " | " ,tfidf_dict[word2])
                    sim = sim + model_w2v2.similarity(word1, word2)*tfidf_dict[word1]*tfidf_dict[word2]
    return sim                        
                    

In [35]:
word_vectors = model_w2v2.wv
sim = findSentenceCorrelation(df.iloc[91482], model_w2v2, X_tfidf, feature_names,word_vectors)
sim

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


0.19640150654939376

In [36]:
#Find cosine_corr between 2 sentences using word2vec and tf-idf
feature_names = tfidf.get_feature_names()
word_vectors = model_w2v2.wv
df['cosine_corr'] = df.apply(lambda x: findSentenceCorrelation(x,model_w2v2, X_tfidf, feature_names, word_vectors), axis=1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [37]:
df.iloc[[91482]]

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5,cosine_corr
91482,contradiction,( ( A dog ) ( ( jumps ( through water ) ) . ) ),( ( The dog ) ( ( is snoring ) . ) ),(ROOT (S (NP (DT A) (NN dog)) (VP (VBZ jumps) ...,(ROOT (S (NP (DT The) (NN dog)) (VP (VBZ is) (...,A dog jumps through water.,The dog is snoring.,3407357681.jpg#4,3407357681.jpg#4r1c,contradiction,NaN,NaN,NaN,NaN,0.196402


In [229]:
df.head()

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5,cosine_corr
0,neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,NaN,NaN,NaN,NaN,0.633051
1,contradiction,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4,3416050480.jpg#4r1c,contradiction,NaN,NaN,NaN,NaN,0.000683
2,entailment,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,"( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4,3416050480.jpg#4r1e,entailment,NaN,NaN,NaN,NaN,0.757841
3,neutral,( Children ( ( ( smiling and ) waving ) ( at c...,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2,2267923837.jpg#2r1n,neutral,NaN,NaN,NaN,NaN,0.320495
4,entailment,( Children ( ( ( smiling and ) waving ) ( at c...,( There ( ( are children ) present ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,Children smiling and waving at camera,There are children present,2267923837.jpg#2,2267923837.jpg#2r1e,entailment,NaN,NaN,NaN,NaN,0.314972


In [ ]:

# from sklearn.feature_extraction.text import TfidfVectorizer
# pass_through = lambda x:x
# tfidf = TfidfVectorizer(analyzer=pass_through)
# # compute tf-idf weights for the words in the training set questions
# X_tfidf = tfidf.fit_transform(X_train_tokens)

In [38]:
from fuzzywuzzy import fuzz

# ratio
df['fuzzy_ratio'] = df.apply(lambda row: fuzz.ratio(str(row['sentence1']), str(row['sentence2'])), axis=1)

# partial ratio
df['fuzzy_partialratio'] = df.apply(lambda row: fuzz.partial_ratio(str(row['sentence1']), str(row['sentence2'])), axis=1)

# token_sort_ratio
df['fuzzy_tokensortratio'] = df.apply(lambda row: fuzz.token_sort_ratio(str(row['sentence1']), str(row['sentence2'])), axis=1)

# token_set_ratio
df['fuzzy_tokensetratio'] = df.apply(lambda row: fuzz.token_set_ratio(str(row['sentence1']), str(row['sentence2'])), axis=1)


/usr/local/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [39]:
df.head()

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5,cosine_corr,fuzzy_ratio,fuzzy_partialratio,fuzzy_tokensortratio,fuzzy_tokensetratio
0,neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,NaN,NaN,NaN,NaN,0.679220,52,51,48,48
1,contradiction,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4,3416050480.jpg#4r1c,contradiction,NaN,NaN,NaN,NaN,0.027683,44,42,52,47
2,entailment,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,"( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4,3416050480.jpg#4r1e,entailment,NaN,NaN,NaN,NaN,0.774539,46,58,57,74
3,neutral,( Children ( ( ( smiling and ) waving ) ( at c...,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2,2267923837.jpg#2r1n,neutral,NaN,NaN,NaN,NaN,0.354233,49,52,54,54
4,entailment,( Children ( ( ( smiling and ) waving ) ( at c...,( There ( ( are children ) present ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,Children smiling and waving at camera,There are children present,2267923837.jpg#2,2267923837.jpg#2r1e,entailment,NaN,NaN,NaN,NaN,0.323046,35,38,48,48


In [40]:
def findWeightedVecRepresentation(row, model_w2v2, X_tfidf, word_vectors, feature_names):
    tfidf_dict = findTfIdf(row.name,X_tfidf, feature_names)
    sent1 = tokenize_sentence(row['sentence1'])
    sent2 = tokenize_sentence(row['sentence2'])
    sim = 0                    
    bucket1 = [0] * 300                              
    bucket2 = [0] * 300                                                            
    for word1 in sent1:
        vec1 = [0] * 300                                                      
        if word1 in word_vectors.vocab:
            vec1 =  model_w2v2[word1]
            for j in range(300):
                bucket1[j] = bucket1[j] + tfidf_dict[word1] * vec1[j]
    for word1 in sent2:
        vec1 = [0] * 300                                                      
        if word1 in word_vectors.vocab:
            vec1 =  model_w2v2[word1]  
            for j in range(300):
                bucket2[j] = bucket2[j] + tfidf_dict[word1] * vec1[j]                         
    return bucket1, bucket2                        
    

In [41]:
def initColumns(df):
    for i in range(300):
        df['{}_vec1'.format(i)] = 0
        df['{}_vec2'.format(i)] = 0

In [42]:
initColumns(df)

In [43]:
df.head()

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,...,295_vec1,295_vec2,296_vec1,296_vec2,297_vec1,297_vec2,298_vec1,298_vec2,299_vec1,299_vec2
0,neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,...,0,0,0,0,0,0,0,0,0,0
1,contradiction,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4,3416050480.jpg#4r1c,contradiction,...,0,0,0,0,0,0,0,0,0,0
2,entailment,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,"( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4,3416050480.jpg#4r1e,entailment,...,0,0,0,0,0,0,0,0,0,0
3,neutral,( Children ( ( ( smiling and ) waving ) ( at c...,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2,2267923837.jpg#2r1n,neutral,...,0,0,0,0,0,0,0,0,0,0
4,entailment,( Children ( ( ( smiling and ) waving ) ( at c...,( There ( ( are children ) present ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,Children smiling and waving at camera,There are children present,2267923837.jpg#2,2267923837.jpg#2r1e,entailment,...,0,0,0,0,0,0,0,0,0,0


In [44]:
word_vectors = model_w2v2.wv
feature_names = tfidf.get_feature_names()
for i, row in df.iterrows():
    bucket1, bucket2 = findWeightedVecRepresentation(row, model_w2v2, X_tfidf, word_vectors, feature_names)
    for j in range(0,300,1):
        df.at[i,'{}_vec1'.format(j)] = bucket1[j]
        df.at[i,'{}_vec2'.format(j)] = bucket2[j]                                             

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [45]:
df.head()

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,...,295_vec1,295_vec2,296_vec1,296_vec2,297_vec1,297_vec2,298_vec1,298_vec2,299_vec1,299_vec2
0,neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,...,0,1,0,0,0,1,0,0,0,1
1,contradiction,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4,3416050480.jpg#4r1c,contradiction,...,0,1,0,0,0,0,0,0,0,0
2,entailment,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,"( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4,3416050480.jpg#4r1e,entailment,...,0,1,0,0,0,0,0,0,0,0
3,neutral,( Children ( ( ( smiling and ) waving ) ( at c...,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2,2267923837.jpg#2r1n,neutral,...,0,0,-1,0,1,0,1,0,0,0
4,entailment,( Children ( ( ( smiling and ) waving ) ( at c...,( There ( ( are children ) present ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,Children smiling and waving at camera,There are children present,2267923837.jpg#2,2267923837.jpg#2r1e,entailment,...,1,2,-1,0,1,0,1,0,0,0


In [46]:
df.iloc[[0]]

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,...,295_vec1,295_vec2,296_vec1,296_vec2,297_vec1,297_vec2,298_vec1,298_vec2,299_vec1,299_vec2
0,neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,...,0,1,0,0,0,1,0,0,0,1


In [47]:
X = df

In [48]:
dropped_cols = ['sentence1_binary_parse','sentence2_binary_parse','sentence1_parse','sentence2_parse',
                'sentence1','sentence2','captionID','pairID','label1','label2','label3','label4','label5']
X = X.drop(dropped_cols, axis = 1) 

In [49]:
X.head()

,gold_label,cosine_corr,fuzzy_ratio,fuzzy_partialratio,fuzzy_tokensortratio,fuzzy_tokensetratio,0_vec1,0_vec2,1_vec1,1_vec2,...,295_vec1,295_vec2,296_vec1,296_vec2,297_vec1,297_vec2,298_vec1,298_vec2,299_vec1,299_vec2
0,neutral,0.679220,52,51,48,48,0,-1,1,0,...,0,1,0,0,0,1,0,0,0,1
1,contradiction,0.027683,44,42,52,47,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,entailment,0.774539,46,58,57,74,0,-1,1,1,...,0,1,0,0,0,0,0,0,0,0
3,neutral,0.354233,49,52,54,54,1,0,0,0,...,0,0,-1,0,1,0,1,0,0,0
4,entailment,0.323046,35,38,48,48,1,0,0,0,...,1,2,-1,0,1,0,1,0,0,0


In [50]:
y_train = X['gold_label']
X_train = X.drop(['gold_label'], axis = 1) 

In [51]:
X_train.head()

,cosine_corr,fuzzy_ratio,fuzzy_partialratio,fuzzy_tokensortratio,fuzzy_tokensetratio,0_vec1,0_vec2,1_vec1,1_vec2,2_vec1,...,295_vec1,295_vec2,296_vec1,296_vec2,297_vec1,297_vec2,298_vec1,298_vec2,299_vec1,299_vec2
0,0.679220,52,51,48,48,0,-1,1,0,-1,...,0,1,0,0,0,1,0,0,0,1
1,0.027683,44,42,52,47,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0.774539,46,58,57,74,0,-1,1,1,-1,...,0,1,0,0,0,0,0,0,0,0
3,0.354233,49,52,54,54,1,0,0,0,-1,...,0,0,-1,0,1,0,1,0,0,0
4,0.323046,35,38,48,48,1,0,0,0,-1,...,1,2,-1,0,1,0,1,0,0,0


In [52]:
y_train.head()

0          neutral
1    contradiction
2       entailment
3          neutral
4       entailment
Name: gold_label, dtype: object

In [63]:
y_train.value_counts()

entailment       183414
contradiction    183185
neutral          182762
-                   785
Name: gold_label, dtype: int64

In [66]:
def encodeY(val):
    if(val.strip() == "entailment"):
        return 0
    elif (val.strip() == "neutral"):
        return 1
    elif (val.strip() == "contradiction"):
        return 2
    else:
        return 3

In [ ]:
y_train['label'] = y_train.apply(lambda row: y_train.encodeY(row['']), axis=1)

In [67]:
y_train_new = [0] * len(y_train)                              
for i in range(len(y_train)):
    y_train_new[i] = encodeY(y_train[i])

In [74]:
y_train_new[3]

1

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
# 5-fold cross-validation for model tuning
logr_model = LogisticRegression(random_state=42)

In [75]:
lr = LogisticRegression(C=1)
lr.fit(X_train, y_train_new)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [78]:
lr.score(X_train,y_train_new)

0.5762852042912245

In [55]:
np.logspace(-2, 7, 10)

array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07])

In [56]:
np.logspace(-5, -1, 5)

array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01])

In [ ]:
param_grid = {'C': np.logspace(-2, 7, 10),
             'tol': np.logspace(-5, -1, 5)
             }
logr_cv = RandomizedSearchCV(logr_model, param_distributions=param_grid, cv=5, n_jobs=-1)
# X_train_features contains all features from feature sets 1,2 & 3 for the training set question-pairs
logr_cv.fit(X_train, y_train_new)

In [ ]:
# train the tuned model
logr_model = LogisticRegression(random_state=42, 
                                C=logr_cv.best_params_['C'], 
                                tol=logr_cv.best_params_['tol'], 
                                n_jobs=-1)
logr_model.fit(X_train, y_train_new)


In [ ]:
#Preparing test data

testdf = pd.read_csv("~/Downloads/snli_1.0/snli_1.0_test.csv", sep='\t')
testdf.head()

In [ ]:

testdf.dropna(subset = ["sentence2"], inplace=True)
testdf = testdf.reset_index(drop=True)

In [ ]:
#Find cosine_corr between 2 sentences using word2vec and tf-idf
feature_names = tfidf.get_feature_names()
word_vectors = model_w2v2.wv
testdf['cosine_corr'] = testdf.apply(lambda x: findSentenceCorrelation(x,model_w2v2, X_tfidf, feature_names, word_vectors), axis=1)

In [ ]:
from fuzzywuzzy import fuzz

# ratio
testdf['fuzzy_ratio'] = testdf.apply(lambda row: fuzz.ratio(str(row['sentence1']), str(row['sentence2'])), axis=1)

# partial ratio
testdf['fuzzy_partialratio'] = testdf.apply(lambda row: fuzz.partial_ratio(str(row['sentence1']), str(row['sentence2'])), axis=1)

# token_sort_ratio
testdf['fuzzy_tokensortratio'] = testdf.apply(lambda row: fuzz.token_sort_ratio(str(row['sentence1']), str(row['sentence2'])), axis=1)

# token_set_ratio
testdf['fuzzy_tokensetratio'] = testdf.apply(lambda row: fuzz.token_set_ratio(str(row['sentence1']), str(row['sentence2'])), axis=1)


In [ ]:
word_vectors = model_w2v2.wv
feature_names = tfidf.get_feature_names()
for i, row in testdf.iterrows():
    bucket1, bucket2 = findWeightedVecRepresentation(row, model_w2v2, X_tfidf, word_vectors, feature_names)
    for j in range(0,300,1):
        testdf.at[i,'{}_vec1'.format(j)] = bucket1[j]
        testdf.at[i,'{}_vec2'.format(j)] = bucket2[j]                                             